### Task C - Producer 2


**Producer 2** is used to read data from the Aqua streaming csv file and send data to the **Apache Spark Streaming** application in a time interval of 10 to 30 seconds (generated randomly). 

Producer is designed to run infinitely to produce sufficient stream of data to the streaming application.


In [ ]:
# import statements
from time import sleep
from json import dumps
from random import *
from kafka import KafkaProducer
import random
import json
import csv
import datetime as dt

In [ ]:
#In this block, csv file is read in a list
hotspot_Aqua=[]

def read_csv(filename):    
    with open(filename,'r') as f:
        reader=csv.reader(f)
        row_count=1
        
        for row in reader:           
            hotspot_Aqua.append(row)
        row_count += 1
            
    return hotspot_Aqua

climate = read_csv('hotspot_AQUA_streaming.csv')
print(hotspot_Aqua[0])



In [ ]:
#This block is to test if the temp dictionary is generated properly
temp={}
i=0
for i in range(len(hotspot_Aqua)-1):
        rand=random.randint(1,len(hotspot_Aqua)-1)
      #  print(rand)
        temp['latitue']=float(hotspot_Aqua[rand][0])
        temp['longitute']=float(hotspot_Aqua[rand][1])
        temp['confidence']=int(hotspot_Aqua[rand][2])
        temp['surf_air_temp']=int(hotspot_Aqua[rand][3])        
        temp['sender_id']='Climate'
        temp['time_stamp']=str(dt.datetime.now().strftime("%X"))
       # print(temp)   
        #print(json.dumps(temp))
        
        data= json.dumps(temp)
        print(i,data)


In [ ]:
#This block has 2 functions, one creates a kafka producer instance and the publish message is used to send 
#the data in string form to the streaming application

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
       # print(''+ str(value))
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()       
        print('Message published successfully. Data: ' + str(value))
       # print(value_bytes)
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
        print('producer instance created')
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    

if __name__ == '__main__':
   
    topic = 'S1'
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    temp={}
    i=0
    # while true is used for the producer to run infinitely
    while True:
    #for i in range(len(hotspot_Aqua)-1):
        rand=random.randint(1,len(hotspot_Aqua)-1)
      #  print(rand)
        temp['latitude']=float(hotspot_Aqua[rand][0])
        temp['longitude']=float(hotspot_Aqua[rand][1])
        temp['confidence']=int(hotspot_Aqua[rand][2])
        temp['surf_air_temp']=int(hotspot_Aqua[rand][3])        
        temp['sender_id']='Aqua'
        temp['time_stamp']=str(dt.datetime.now().strftime("%X"))
       # print(temp)   
        #print(json.dumps(temp))
        data= json.dumps(temp)
       # print(i,data)
       # print(data)
        key='P2'
        publish_message(producer, topic, key, data)
       
        
        sleep(randint(10,30))